# Optimización de parámetros sobre el modelo Decision Tree

El objetivo de este proyecto corto se centra en optimizar los parámetros de los algoritmos utilizados en el proyecto anterior (Modelos de Predicción), por ello se sigue trabajando con el dataset filtrado y transformado de propiedades en venta publicadas en el portal [Properati](www.properati.com.ar).

La métrica usada para medir es el RMSE (raíz del error cuadrático medio), cuya fórmula es:

$$RMSE = \sqrt{\frac{\sum_{t=1}^n (\hat y_t - y_t)^2}{n}}$$

**1.** Se importa la librería Pandas para leer el dataset.

In [1]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
path_dataset = 'dataset/datos_properati_limpios_model.csv'
df = pd.read_csv(path_dataset)

**2.** Se divide el dataset en un conjunto de entrenamiento (80%) y un conjunto de test (20%) utilizando como target la columna 'price_aprox_usd'.

In [2]:
from sklearn.model_selection import train_test_split
import numpy as np
np.random.seed(123)
X = df.drop(['price_aprox_usd'], axis=1)
y = df['price_aprox_usd']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=33)

In [3]:
print(X_train.shape[0], X_test.shape[0])

5100 1276


### Entrenamiento con Scikit-learn 
- Para realizar un cross validation es necesario definir la cantidad de folds, en este caso se usan cinco.

- GridSearchCV permite testear, a través de un espacio de búsqueda de parámetros la mejor combinación posible dado un estimador.

- Por ejemplo, en este caso se prueba la profundidad máxima y la máxima cantidad de features para hacer los split, ambos entre 1 y 5.

**3.** Se crea una variable 'param_grid' con valores del 1 al 5 para los atributos 'max_depth' y 'max_features'. 

In [4]:
from sklearn.model_selection import GridSearchCV
param_grid = [{'max_depth': [1, 2, 3, 4, 5], 'max_features': [1, 2, 3, 4, 5]}]

**4.** Se importan GridSearchCV y DecisionTreeRegressor desde Scikit-learn y se crea la variable 'grid_search' asignando a esta un 'GridSearchCV' que recorre el 'param_grid' creado con el algoritmos DecisionTreeRegressor y el 'neg_mean_squared_error' como scoring.

In [5]:
from sklearn.tree import DecisionTreeRegressor
tree = DecisionTreeRegressor()
grid_search = GridSearchCV(tree, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)

**5.** A continuación, se realiza el 'fit' del 'grid_search' sobre el conjunto de entrenamiento.

In [6]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'max_depth': [1, 2, 3, 4, 5], 'max_features': [1, 2, 3, 4, 5]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

**6.** Se muestran los resultados (grid_scores) obtenidos durante el 'grid_search'.

In [7]:
(grid_search.cv_results_['mean_train_score'])

array([-9.51101896e+08, -9.48947123e+08, -9.61452946e+08, -9.09160633e+08,
       -9.61645759e+08, -9.16301633e+08, -9.10607358e+08, -9.05051866e+08,
       -9.02027996e+08, -9.17910707e+08, -9.34057111e+08, -8.95991684e+08,
       -8.39885841e+08, -7.64112369e+08, -8.46945982e+08, -9.47462245e+08,
       -9.28434274e+08, -8.43431343e+08, -6.84134116e+08, -8.58542609e+08,
       -8.31164840e+08, -8.00984506e+08, -7.60910219e+08, -7.84383768e+08,
       -8.18491466e+08])

**7.** Se muestra el mejor score y los mejores parámetros encontrados por el estimador.

In [8]:
print(grid_search.best_estimator_)
print('El mejor score es:', (grid_search.best_score_))
print('Los mejores parámetros son:', grid_search.best_params_)

DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=4,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')
El mejor score es: -699214958.42845
Los mejores parámetros son: {'max_depth': 4, 'max_features': 4}


Segun el estimador, el valor con mejor resultado (dado el espacio de búsqueda definido) es la combinación de max_depth=4 y max_features=4.

**8.** Convertimos NMSE a RMSE.

In [9]:
def nmsq2rmse(score):
    return np.round(np.sqrt(-score), 2)

In [10]:
print('El mejor RMSE es:', nmsq2rmse(grid_search.best_score_))

El mejor RMSE es: 26442.67


**9.** En este punto, se realiza el mismo procedimiento anterior con el fin de encontrar el mejor modelo, para el espacio de búsqueda definido por los siguientes parametros adicionales:

* "min_samples_split": [2, 10, 20]
* "max_depth": [None, 2, 5, 10, 15]
* "min_samples_leaf": [1, 5, 10, 15]
* "max_leaf_nodes": [None, 5, 10, 20]

In [11]:
param_grid = [{'max_depth': [None, 2, 5, 10, 15], 'min_samples_split': [2, 10, 20], 'min_samples_leaf': [1, 5, 10, 15], 'max_leaf_nodes': [None, 5, 10, 20]}]

In [12]:
grid_search = GridSearchCV(tree, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'max_depth': [None, 2, 5, 10, 15], 'min_samples_split': [2, 10, 20], 'min_samples_leaf': [1, 5, 10, 15], 'max_leaf_nodes': [None, 5, 10, 20]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

In [13]:
print(grid_search.best_estimator_)
print('El mejor score es:', np.sqrt(-grid_search.best_score_))
print('Los mejores parámetros son:', grid_search.best_params_)

DecisionTreeRegressor(criterion='mse', max_depth=15, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=15,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')
El mejor score es: 21373.445786543256
Los mejores parámetros son: {'max_depth': 15, 'max_leaf_nodes': None, 'min_samples_leaf': 15, 'min_samples_split': 2}


**10.** Se guarda el mejor modelo resultante como 'optimised_decision_tree'.

In [14]:
optimised_decision_tree = grid_search.best_estimator_
optimised_decision_tree

DecisionTreeRegressor(criterion='mse', max_depth=15, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=15,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

__11.__ Se evalua el desempeño de este modelo en el conjunto de testing.

In [15]:
from sklearn.metrics import mean_squared_error
y_opt_pred = optimised_decision_tree.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_opt_pred))
np.round(rmse)

20853.0

**12.** Se observan los primeros diez resultados de la predicción sobre el valor de las propiedades.

In [16]:
val_real = pd.Series(y_test.values)
val_pred = pd.Series(y_opt_pred)

In [17]:
predicciones = pd.concat([val_real.rename('Valor real'),val_pred.rename('Valor Pred') ,abs(val_real-val_pred).rename('Dif(+/-)')] ,  axis=1)

In [18]:
predicciones.head(10)

,Valor real,Valor Pred,Dif(+/-)
0,87000.000,82738.019,4261.981
1,125000.000,112310.345,12689.655
2,145000.000,120261.905,24738.095
3,68700.000,84819.048,16119.048
4,176510.000,135189.315,41320.685
5,115000.000,82333.333,32666.667
6,84000.000,97062.750,13062.750
7,135000.000,131846.125,3153.875
8,118000.000,111119.637,6880.363
9,80000.000,82860.000,2860.000
